In [ ]:
#### IMPORTING THE LIBERARIES REQUIRED ####

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

In [ ]:
#### CONVERTING THE IMAGES TO ARRAYS ####

training_data = []
DATADIR = r"C:\Users\bvraju\Desktop\AlexNet\PetImages"
CATEGORIES = ["Dog","Cat"]
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(227,227))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()

In [ ]:
#### SAVING THE LABELS TO FILE ####

import pickle
pickle_out = open('LABELS_FOR_COLOUR_IMAGES.pickle','wb')
pickle.dump(X_data, pickle_out)
pickle_out.close()

In [1]:
#### LOADING THE x AND y LABELS ####

import pickle  
temp = open('LABELS_FOR_COLOUR_IMAGES.pickle','rb')
X_y = pickle.load(temp)
temp.close()
X_data=[]
y_data=[]
for x,y in X_y:
    X_label = x
    y_label = y
    X_data.append(X_label)
    y_data.append(y_label)
print("total number of examples : ",len(X_y))

In [4]:
#### CONVERTING THE LIST OF LABELS TO ARRAY ####

import numpy as np
X=np.array(X_data).reshape(-1,227,227,3)
y=np.array(y_data).reshape(-1,1)
print("shape of the input array ",X.shape)
print("shape of the output label ",y.shape)

In [6]:
#### SPLLITING THE DATA INTO TRAIN (training set includes validation set) AND TEST SET ####

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2) 
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(19956, 227, 227, 3) (4990, 227, 227, 3) (19956, 1) (4990, 1)


In [8]:

##### ALEXNET MODEL ####

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

def alexnet():
    model = Sequential()
    model.add(Conv2D(96, (11, 11),strides=4, padding='valid', input_shape=(227,227,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    model.add(Conv2D(256, (5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    model.add(Conv2D(384, (3, 3), padding='same'))
    model.add(Conv2D(384, (3, 3), padding='same'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    model.add(Flatten())
    model.add(Dense(9216))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(4096))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))

    return model

In [9]:
model = alexnet() # creating the AlexNet model.

In [10]:
model.summary()  # overview of the model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
activation_2 (Activation)    (None, 27, 27, 256)      

In [11]:
#### TRAINING THE MODEL AND TESTING IT ON VALIDATION SET (validation_split=0.10) ####

model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])
hist=model.fit(X_train,y_train,batch_size=32,epochs=1,validation_split=0.10) 

Train on 17960 samples, validate on 1996 samples
Epoch 1/1
17960/17960 [==============================] - 2306s 128ms/step - loss: 0.5656 - accuracy: 0.7076 - val_loss: 0.5439 - val_accuracy: 0.7199


In [12]:
#### SAVING THE TRAINED WEIGHTS (helps to avoid multiple times of training the model) ####

sure = int(input("press 1 to continue..."))
if sure == 1:
    fname = "AlexNet_weights_TRAINED_WEIGHTS"
    model.save_weights(fname , overwrite=True)
    print("saved the new weights")
else:
    print("no new weights are saved")    

press 1 to continue...1
saved the new weights


In [13]:
#### LOADING THE WEIGHTS(this step can be skipped) ####

fname = "AlexNet_weights_TRAINED_WEIGHTS"
model.load_weights(fname)

In [17]:
#### PREDICTING THE CLASSES FOR THE TEST SET ####

predicted=model.predict_classes([X_test])

print("prediction on ", len(predicted)," images.")

prediction on  4990  images.


In [18]:
#### PRINTTING THE ACCURACY ON THE TESTING SET ####

count=0
for i in range(len(predicted)):
    if predicted[i]==y_test[i]:
        count+=1
print("accuracy on test : ",count/len(X_test))

accuracy on test :  0.7280561122244489
